In [1]:
!pip install -q --no-cache-dir torch-geometric

In [2]:
import cv2
import numpy as np
import pandas as pd
import os
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
from typing import Tuple
     
from scipy.spatial.distance import cdist
from skimage.measure import regionprops_table, label
from skimage.color import rgb2gray
from skimage.measure import regionprops
from skimage import exposure
from skimage.util import view_as_blocks

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
import torch_geometric.nn as geom_nn
from torch_geometric.nn import GCNConv, global_mean_pool
from torchvision import models
from torch.utils.data import DataLoader, Dataset, random_split
from torch_geometric.data import Data, Batch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch_geometric.utils import from_networkx
from torchvision import datasets, transforms
from torch.nn.parallel import DataParallel

import networkx as nx

## **Preprocesing and Model Functions**

In [3]:
def preprocess_image(image, label):
    # Convert to float and rescale to [0, 1]
    image = image.float() / 255.0
    return image, label

def sampling(z_mean, z_log_var):
    epsilon = torch.randn_like(z_mean)
    return z_mean + torch.exp(0.5 * z_log_var) * epsilon

In [4]:
class Encoder1(nn.Module):
    def __init__(self):
        super(Encoder1, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        return x

In [5]:
class Encoder2(nn.Module):
    def __init__(self):
        super(Encoder2, self).__init__()
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv_mean = nn.Conv2d(256, 128, kernel_size=3, padding=1)
        self.conv_log_var = nn.Conv2d(256, 128, kernel_size=3, padding=1)

    def forward(self, x):
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = F.relu(self.conv4(x))
        x = self.pool(x)
        z_mean = self.conv_mean(x)
        z_log_var = self.conv_log_var(x)
        return z_mean, z_log_var

In [6]:
class Decoder1(nn.Module):
    def __init__(self):
        super(Decoder1, self).__init__()
        self.deconv1 = nn.ConvTranspose2d(128, 256, kernel_size=3, padding=1)
        self.deconv2 = nn.ConvTranspose2d(256, 128, kernel_size=3, padding=1)
        self.upsample = nn.Upsample(scale_factor=2)

    def forward(self, x):
        x = F.relu(self.deconv1(x))
        x = self.upsample(x)
        x = F.relu(self.deconv2(x))
        x = self.upsample(x)
        return x

In [7]:
class Decoder2(nn.Module):
    def __init__(self):
        super(Decoder2, self).__init__()
        self.deconv3 = nn.ConvTranspose2d(128, 64, kernel_size=3, padding=1)
        self.deconv4 = nn.ConvTranspose2d(64, 32, kernel_size=3, padding=1)
        self.output_layer = nn.ConvTranspose2d(32, 3, kernel_size=3, padding=1)
        self.upsample = nn.Upsample(scale_factor=2)

    def forward(self, x):
        x = F.relu(self.deconv3(x))
        x = self.upsample(x)
        x = F.relu(self.deconv4(x))
        x = self.upsample(x)
        x = torch.sigmoid(self.output_layer(x))
        return x

In [8]:
class VAE(nn.Module):
    def __init__(self, encoder1, encoder2, decoder1, decoder2):
        super(VAE, self).__init__()
        self.encoder1 = encoder1
        self.encoder2 = encoder2
        self.decoder1 = decoder1
        self.decoder2 = decoder2

    def forward(self, x):
        x = self.encoder1(x)
        z_mean, z_log_var = self.encoder2(x)
        z = sampling(z_mean, z_log_var)
        x = self.decoder1(z)
        reconstructed = self.decoder2(x)
        return reconstructed, z_mean, z_log_var

## **DataLoader**

In [9]:
# Define parameters
root_dir = "/kaggle/input/hepatoma-data/Histopathology-Images"
batch_size = 4
img_size = 1600
prefetch_size = 2  # This will translate to `num_workers` in PyTorch
train_split_ratio = 0.8  # 80% for training, 20% for testing

# Image preprocessing: Resizing and normalization (to [0, 1])
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor()  # Converts images to tensor and scales them to [0, 1]
])

# Create the full dataset
full_dataset = datasets.ImageFolder(root=root_dir, transform=transform)

# Calculate split sizes
train_size = int(train_split_ratio * len(full_dataset))
test_size = len(full_dataset) - train_size

# Split the dataset into training and testing sets
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Create DataLoaders for training and testing sets
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=prefetch_size, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=prefetch_size, pin_memory=True)

## **Training and Testing Steps**

In [10]:
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               optimizer: torch.optim.Optimizer,
               device: torch.device,
               recon_factor: float = 1.0,
               kl_factor: float = 1.0) -> float:
    model.train()
    train_loss = 0

    for batch, (X, _) in tqdm(enumerate(dataloader), total=len(dataloader)):
        X = X.to(device)

        # Forward pass
        reconstructed, z_mean, z_log_var = model(X)

        # Calculate loss
        loss = vae_loss(reconstructed, X, z_mean, z_log_var, recon_factor, kl_factor)
        train_loss += loss.item()

        # Zero gradients
        optimizer.zero_grad()

        # Backward pass
        loss.backward()

        # Optimizer step
        optimizer.step()

    # Average loss per batch
    train_loss /= len(dataloader)
    return train_loss


def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              device: torch.device,
              recon_factor: float = 1.0,
              kl_factor: float = 1.0) -> float:
    model.eval()
    test_loss = 0

    with torch.no_grad():
        for batch, (X, _) in tqdm(enumerate(dataloader), total=len(dataloader)):
            X = X.to(device)

            # Forward pass
            reconstructed, z_mean, z_log_var = model(X)

            # Calculate loss
            loss = vae_loss(reconstructed, X, z_mean, z_log_var, recon_factor, kl_factor)
            test_loss += loss.item()

    # Average loss per batch
    test_loss /= len(dataloader)
    return test_loss

## **Training Loop**

In [11]:
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          epochs: int,
          device: torch.device,
          recon_factor: float = 1.0,
          kl_factor: float = 1.0) -> dict:

    results = {"train_loss": [],
               "test_loss": []}

    model.to(device)

    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")

        # Training step
        train_loss = train_step(model=model,
                                dataloader=train_dataloader,
                                optimizer=optimizer,
                                device=device,
                                recon_factor=recon_factor,
                                kl_factor=kl_factor)

        # Testing step
        test_loss = test_step(model=model,
                              dataloader=test_dataloader,
                              device=device,
                              recon_factor=recon_factor,
                              kl_factor=kl_factor)

        print(f"Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f}")

        # Store results
        results["train_loss"].append(train_loss)
        results["test_loss"].append(test_loss)

    return results

## **Training**

In [12]:
def vae_loss(reconstructed, original, z_mean, z_log_var, recon_factor=1.0, kl_factor=1.0):
    # Reconstruction loss (MSE)
    reconstruction_loss = nn.functional.mse_loss(reconstructed, original, reduction='sum')
    
    # KL divergence loss
    kl_loss = -0.5 * torch.sum(1 + z_log_var - z_mean.pow(2) - z_log_var.exp())
    
    # Weighted sum of both losses
    return recon_factor * reconstruction_loss + kl_factor * kl_loss

In [13]:
# Check for available GPUs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [14]:
# Initialize the model and move it to the device
encoder1 = Encoder1().to(device)
encoder2 = Encoder2().to(device)
decoder1 = Decoder1().to(device)
decoder2 = Decoder2().to(device)
vae = VAE(encoder1, encoder2, decoder1, decoder2).to(device)

In [15]:
# Initialize optimizer
optimizer = torch.optim.Adam(vae.parameters(), lr=1e-4)

In [16]:
# Optionally set up multi-GPU training
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    vae = DataParallel(vae)

# Number of GPUs being used
num_devices = torch.cuda.device_count()
print(f"Number of devices: {num_devices}")

Using 2 GPUs
Number of devices: 2


In [17]:
# Train the model and store the results in "history" variable
history = train(
    model=vae,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    optimizer=optimizer,
    epochs=256,
    device=device,
    recon_factor=1.0,  # Custom weight for reconstruction loss
    kl_factor=0      # Custom weight for KL divergence
)     

Epoch 1/256


  0%|          | 0/135 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
100%|██████████| 34/34 [00:09<00:00,  3.58it/s]

Train Loss: 559776.6056 | Test Loss: 364430.4439
Epoch 2/256



100%|██████████| 34/34 [00:08<00:00,  3.85it/s]

Train Loss: 229940.1732 | Test Loss: 175648.5551
Epoch 3/256


...

Train Loss: 13716.8315 | Test Loss: 14416.8218
Epoch 255/256



100%|██████████| 34/34 [00:08<00:00,  3.95it/s]

Train Loss: 14527.9317 | Test Loss: 14951.0263
Epoch 256/256



100%|██████████| 34/34 [00:09<00:00,  3.77it/s]

Train Loss: 13826.6195 | Test Loss: 13627.3053


In [20]:
# Save Model
torch.save(
        obj=vae.state_dict(),
        f=f"/kaggle/working/vae_512.pth"
)

In [22]:
history_df = pd.DataFrame(history)
history_df.to_csv("/kaggle/working/history-vae_512.csv")